In [1]:
%%capture
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import WeightedRandomSampler
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil
import pandas as pd
import random
import os
import pickle
import transformers
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AdamW
import heapq
from scipy.stats import spearmanr
import itertools

# Preparing Data

In [2]:
def download_data():
    
  %cd $root
  if os.path.isdir(path_data) == False:
    os.makedirs(path_data)
  %cd $path_data

  # Download your kaggle coockies file and put it in root directory
  !cp ../kaggle.json /root/.kaggle/
  !kaggle competitions download -c stat946winter2021dc2
  !unzip train.pkl.zip
  !unzip test.pkl.zip

def pkl_to_dict():

  %cd $path_data
  
  FILE_NAME = 'train.pkl'
  infile = open(FILE_NAME,'rb')
  trainset = pickle.load(infile)


  FILE_NAME = 'test.pkl'
  infile = open(FILE_NAME,'rb')
  testset = pickle.load(infile)

  return trainset, testset

def load_text(train_set, test_set):
  
  # Train sentences in the correct order
  train_texts = []
  test_texts = []
  labels = []


  # Concatenate the sentences in each paragraph
  for parag in train_set:
    # A single paragraph
    p = ""
    for i in range(len(parag['sentences'])):
      if i != 0:
        p += parag['sentences'][i] + ' [SEP] '
      else:
        p += ' [CLS] ' + parag['sentences'][i] + ' [SEP] '
    
    train_texts.append(p)
    labels.append(parag["indexes"])


  for parag in test_set:
    # A single paragraph
    p = ""
    for i in range(len(parag['sentences'])):
      if i != 0:
        p +=  parag['sentences'][i] + ' [SEP] '
      else:
        p += ' [CLS] ' + parag['sentences'][i] + ' [SEP] '
    
    test_texts.append(p)
  
  return train_texts, labels, test_texts




In [3]:
root = %pwd # To Do replace the path with %pwd
path_data = os.path.join(root, "data/")

In [4]:
# # Download the datast To Do
# # download_data()

# Reading the data from pkl file
train_set, test_set = pkl_to_dict()

# Get list of paragraphs in form of a single string
train_texts, labels, test_texts = load_text(train_set[:280000], test_set)

# # redcuing the volume of data
# train_texts = train_texts[:180000]
# labels = labels[:180000]

# # Seperating 10% of train data for validation

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, labels, test_size=.1)

/home/m25dehgh/testing_codes/stat/dc1/data


In [5]:
len(train_texts)

252000

# Tokenizing

In [33]:
def remove_big_parags(train_encodings):

  # Removing paragraphs bigger than 512 token
  i = 0
  while i != len(train_encodings['attention_mask']):
    
    if train_encodings['attention_mask'][i][-1] == 1:
      del train_encodings['input_ids'][i]
      del train_encodings['attention_mask'][i]
      i -= 1

    i += 1

  
  return train_encodings


def add_token_type_sep(encodings, desired_sep_nums=6):
  '''
    This function adds two new lists to the encoding ditionary: token_type_ids and sep_indxs # To Do
    token_type_ids: list of size [encodings len, paragrraph len] 
    sep_indxs: list of size [encoding len, number of [SEP] tokens in the paragraph]. This list contian [SEP] indices.

  '''
  # The list of token types for all paragraphs
  token_types = []
  sep_indxs = []

  for p_indx, parag in enumerate(encodings['input_ids']):
    curr_type = 0
    types_list = []
    sep_toks = []

    for i, tok in enumerate(parag):
      types_list.append(curr_type)

      if tok == 2:
        sep_toks.append(i)
        curr_type = 1 - curr_type # Flipping the token type

    
    if(len(sep_toks) < desired_sep_nums):
      for i in range(desired_sep_nums - len(sep_toks)):
        encodings['input_ids'][p_indx][-(i+1)] = 2
        sep_toks.append(len(encodings['input_ids'][p_indx]) - (i+1))

    sep_indxs.append(sep_toks)
    token_types.append(types_list)
    
  
  encodings['token_type_ids'] = token_types
  encodings['sep_indxs'] = sep_indxs
      
  return encodings


def save_obj(obj, path='train'):
  
  %cd $path_data

  with open(path + ".txt", "wb") as fp:
    pickle.dump(obj, fp)




def load_saved_obj(path='train'):

  %cd $path_data

  with open(path + ".txt", "rb") as fp:
    obj = pickle.load(fp)


  return obj

def seq_to_out(seq):
    out = []
    if type(seq[0]) != list:
        if type(seq) == str:
            seq = [int(i) for i in seq]
        for i in range(6):
            out.append(seq.index(i))
    else:
        for each_seq in seq:
            new_seq = []
            for i in range(6):
                new_seq.append(each_seq.index(i))
            out.append(new_seq)
    return out

In [5]:
tokenizer = transformers.DebertaTokenizer.from_pretrained('microsoft/deberta-base') 

In [6]:
# train_encodings = tokenizer(train_texts, add_special_tokens=False, truncation=True, padding=True, return_token_type_ids=False) 
# test_encodings = tokenizer(test_texts, add_special_tokens=False, truncation=True, padding=True, return_token_type_ids=False) 
# val_encodings = tokenizer(val_texts, add_special_tokens=False, truncation=True, padding=True, return_token_type_ids=False) 


# Removing paragraphs bigger than 512 token
print("removing big parags from train data")
# train_encodings = remove_big_parags(train_encodings)


# Adding the token_type_ids lists and list of [SEP] postions to to the encodings dictionaries
print("adding token types and [SEP] postions list")
# train_encodings = add_token_type_sep(train_encodings)
# test_encodings = add_token_type_sep(test_encodings)
# val_encodings = add_token_type_sep(val_encodings)

print("saving")
# save_obj(train_encodings, path='train_encodings_280k_FullPar')
# save_obj(train_labels, path='train_labels_280k_FullPar')
# save_obj(val_encodings, path='val_encodings_280k_FullPar')
# save_obj(val_labels, path='val_labels_280k_FullPar')
# save_obj(test_encodings, path='test_encodings_FullPar')

print('loading saved tokenized train encoding dictionary')
train_encodings = load_saved_obj(path='train_encodings_280k_FullPar')
train_labels = load_saved_obj(path='train_labels_280k_FullPar')
val_encodings = load_saved_obj(path='val_encodings_280k_FullPar')
val_labels = load_saved_obj(path='val_labels_280k_FullPar')
test_encodings = load_saved_obj(path='test_encodings_FullPar')

removing big parags from train data
adding token types and [SEP] postions list
saving
loading saved tokenized train encoding dictionary
/home/m25dehgh/testing_codes/stat/dc1/data
/home/m25dehgh/testing_codes/stat/dc1/data
/home/m25dehgh/testing_codes/stat/dc1/data
/home/m25dehgh/testing_codes/stat/dc1/data
/home/m25dehgh/testing_codes/stat/dc1/data


In [35]:
seq_to_out(val_labels[:2])
# type(val_labels[:10][0]) == list

[[3, 4, 2, 5, 0, 1], [0, 1, 2, 5, 3, 4]]

# Creating Dataset

In [36]:
class NSPDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels != None:
          item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # return len(self.labels)
        return len(self.encodings['input_ids'])

In [37]:
train_dataset = NSPDataset(train_encodings, seq_to_out(train_labels))
val_dataset = NSPDataset(val_encodings, seq_to_out(val_labels))
test_dataset = NSPDataset(test_encodings)

In [12]:


# for i in val_encodings['sep_indxs'][1100]:
#   print(val_encodings['input_ids'][1100][i])
# train_encodings['sep_indxs'][1100]

# test_dataset[10]

In [13]:
# m = torch.nn.Softmax(dim=2)
# ml = torch.nn.LogSoftmax(dim=2)
# x = torch.rand(5,15,5)
# print(torch.log(m(x))[2][10].sum())
# ml(x)[2][10].sum()

# Creating Customized NSP Model

In [14]:

# model = NSPDeBerta(sent_nums=3)
# model.to(device)
# model.train()

# sent = ['[CLS] hello [SEP] akbar [SEP] how are [SEP] you.', '[CLS] This is  [SEP] a completely [SEP] She different beautiful [SEP]', ]
# toks = tokenizer(sent, add_special_tokens=False, truncation=True, padding=True, return_token_type_ids=False)
# toks = add_token_type_sep(toks)
# toks = NSPDataset(toks)
# tok_loader = DataLoader(toks, batch_size=2, shuffle=True)
# for toks in tok_loader:
#   input_ids = toks['input_ids'].to(device)
#   attention_mask = toks['attention_mask'].to(device)
#   # labels = toks['labels'].to(device)
#   sep_pos=toks['sep_indxs']
#   token_type_ids=toks['token_type_ids']
#   input_ids
#   outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, sep_pos=sep_pos)

In [15]:
# outputs

In [16]:

# # toks['token_type_ids'] = torch.tensor(toks['token_type_ids'], dtype=torch.float)
# # toks['sep_indxs'] = torch.tensor(toks['sep_indxs'])
# print(toks)
# # from transformers import DebertaTokenizer, DebertaModel
# # import torch
# # tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# # model = DebertaModel.from_pretrained('microsoft/deberta-base')
# # inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(toks['input_ids'], attention_mask=toks['attention_mask'], token_type_ids=toks['token_type_ids'], sep_pos=toks['sep_indxs'])
# last_hidden_states = outputs.last_hidden_state


In [17]:
# print(last_hidden_states.shape)
# sep_pos = [i + j * 17 for j , i in enumerate(toks['sep_indxs'].numpy())]
# sep_pos = torch.tensor(sep_pos)
# last_hidden_states.view(-1,768)[sep_pos.view(-1)].view(2,-1 ,768).shape
# sep_pos

In [ ]:
brgik = False
for i in range(len(test_dataset)):
  for key, val in test_dataset[i].items():
    if key == 'sep_indxs' and val[-1] == 510:
      print('akbaaar error at', i)
      print(val)
      print(test_dataset[i]['input_ids'])
      # brgik = True
      # break
  if brgik:
    break

In [10]:
conf = transformers.DebertaConfig.from_pretrained('microsoft/deberta-base')
conf.hidden_size

768

In [38]:

class NSPDeBerta(torch.nn.Module):
  def __init__(self, sent_nums):
    """
    To Do

    """
    
    super(NSPDeBerta, self).__init__()
    self.sent_nums = sent_nums
    self.DeBerta = transformers.DebertaModel.from_pretrained('microsoft/deberta-base')
    self.conf = transformers.DebertaConfig.from_pretrained('microsoft/deberta-base')
    self.linear1 = torch.nn.Linear(self.conf.hidden_size, self.conf.hidden_size)
    self.linear2 = torch.nn.Linear(self.conf.hidden_size, self.sent_nums)
    self.tan = torch.nn.Tanh()
    self.log_soft = torch.nn.LogSoftmax(dim=2)

  def forward(self, input_ids, attention_mask, token_type_ids, sep_pos):
    """
    To Do
    
    """
    last_hidden_state = self.DeBerta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).last_hidden_state
    
    seq_len = input_ids.shape[1]
    batch_size = input_ids.shape[0]

    # Updating indices of [SEP] in a flatten 2-D matrixx of size (batch_size * seq_len) x (conf.hidden_size)
    sep_batch_flatten_pos = [indx + j * seq_len for j , indx in enumerate(sep_pos.cpu().numpy())]
    sep_batch_flatten_pos = torch.tensor(sep_batch_flatten_pos).to(device)

    # The number of [SEP] positions is qeual to the number of sentences (self.sent_nums)
    # Converting last hidden state of size (batch_size x seq_len x self.conf.hidden_size) to the last hidden states of [SEP] tokens (batch_size, number of sep tokens in each batch , self.conf.hidden_size)
    sep_last_hidden_states = last_hidden_state.view(-1,self.conf.hidden_size)[sep_pos.view(-1)].view(batch_size, self.sent_nums , self.conf.hidden_size).to(device)

    x = self.linear1(sep_last_hidden_states)
    x = self.tan(x)
    x = self.linear2(x)
    
    # Returning the probabilty of each of paraphraph positions for each sentence in each paraphraph
    return self.log_soft(x)

  
  def save_model(self, loss, path='NSPDeberta',):
  
    %cd $root

    model_path = os.path.join(root, 'BestModel/')
    if os.path.isdir(model_path) == False:
      os.makedirs(model_path)
    
    %cd %model_path

    torch.save(self.state_dict(), path + 'loss_' + str(loss))
    




# Training

In [39]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_sents = 6
batch_size = 8

model = NSPDeBerta(num_sents)

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  model = nn.DataParallel(model)


model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

optim = AdamW(model.parameters(), lr=1e-5)

Let's use 3 GPUs!


In [40]:
losses = []

criteria = torch.nn.NLLLoss()

epochs = 1

for epoch in range(epochs):
    print("=================================")
    print("epoch : ", epoch)

    with tqdm(total=int(len(train_dataset) / batch_size)) as pbar:

        for iteration, batch in enumerate(train_loader):
            pbar.update(1)
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            sep_pos=batch['sep_indxs'].to(device)
            token_type_ids=batch['token_type_ids'].to(device)
            output = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, sep_pos=sep_pos)
            loss = torch.tensor([0], requires_grad=True, dtype=torch.float).to(device)

            for b in range(batch_size):
                loss += criteria(output[b], labels[b])
            loss /= batch_size
            losses.append(loss.item())
            loss.backward()
            optim.step()
            torch.cuda.empty_cache()

            if iteration % 100 == 0:
                mean = np.array(losses).mean()
                print("iteration: {} - loss: {}".format(iteration, mean))
                losses.clear() 

# model.save_model(losses[-1], 'NSPDeberta_batch' + str(batch_size) + '_')

model.eval()

epoch :  0


<ipython-input-40-2a88c05a87d1>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=int(len(train_dataset) / batch_size)) as pbar:


iteration: 0 - loss: 1.7957950830459595
iteration: 100 - loss: 1.8009820902347564
iteration: 200 - loss: 1.794909782409668
iteration: 300 - loss: 1.7896342253684998
iteration: 400 - loss: 1.7842701363563538
iteration: 500 - loss: 1.78001478433609
iteration: 600 - loss: 1.7701278674602507
iteration: 700 - loss: 1.761880693435669
iteration: 800 - loss: 1.7446945595741272
iteration: 900 - loss: 1.7339966630935668
iteration: 1000 - loss: 1.7238178956508636
iteration: 1100 - loss: 1.7226910209655761
iteration: 1200 - loss: 1.7137366402149201
iteration: 1300 - loss: 1.7081771719455718
iteration: 1400 - loss: 1.7140716207027435
iteration: 1500 - loss: 1.7047246038913726
iteration: 1600 - loss: 1.6941191959381103
iteration: 1700 - loss: 1.7018040549755096
iteration: 1800 - loss: 1.6984289193153381
iteration: 1900 - loss: 1.6871208047866821
iteration: 2000 - loss: 1.686605749130249
iteration: 2100 - loss: 1.692801467180252
iteration: 2200 - loss: 1.694716637134552
iteration: 2300 - loss: 1.6793

KeyboardInterrupt: 

In [ ]:
bat = next(iter(train_loader))
input_ids = bat['input_ids'].to(device)
torch.cuda.memory_allocated(0)


555086336

In [12]:
torch.cuda.empty_cache()

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0) 
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
a

14593946112

In [ ]:
s = 0
for i in range(len(train_encodings['attention_mask'])):
  s += train_encodings['attention_mask'][i][-1] == 1
s

0

In [ ]:
for iteration, batch in enumerate(train_loader):
  print(iteration)
  print(len(batch))
  print(batch['labels'].shape)
  print(batch['input_ids'].shape)
  # len(bat[10]['labels'])

In [ ]:
# a = tokenizer(train_texts[0:1000], truncation=True, padding=True)
# print(train_texts[0])
# test_text[0]
# train_tex

s = 0
for i in range(1000):
  s += a['attention_mask'][i][-1] == 0
s

996

In [ ]:
len(train_encodings['attention_mask'])

531203

In [ ]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13021         513       10621           0        1886       12232
Swap:             0           0           0


In [ ]:
sent = ['[CLS] hello [SEP] akbar khan', '[CLS] hello [SEP] I want to go home go go go [SEP] She is fucking beautiful [SEP]', ]
toks = tokenizer(sent, add_special_tokens=False, truncation=True, padding=True, return_token_type_ids=False)
tok_type = tokenizer.create_token_type_ids_from_sequences(toks['input_ids'][0], toks['input_ids'][1])
print(tok_type)
# tokenizer.convert_ids_to_tokens(toks)
# print(toks['input_ids'],'\n', toks['attention_mask'], '\n',toks['token_type_ids'])
toks = add_token_type_sep(toks)
print(toks['input_ids'],'\n', toks['attention_mask'], '\n',toks['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[1, 42891, 2, 677, 4901, 449, 4134, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 42891, 2, 100, 236, 7, 213, 184, 213, 213, 213, 2, 2515, 16, 23523, 2721, 2]] 
 [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]] 
 [[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]


In [ ]:
# !rm -rf test.pkl.zip
# !rm -rf train.pkl.zip
# !rm -rf sample_submission.csv
# download_data()

In [2]:
len(test_set)


NameError: name 'test_set' is not defined

In [ ]:
print(train_texts[0])
train_set[10]

['He served in the Oregon House of Representatives from 2009 to 2014, representing inner Southeast and Northeast Portland, and on the County Commission for Multnomah County, Oregon from June 2014 to December 2016.', 'In 2016, Bailey ran for mayor of Portland, losing to Ted Wheeler.', 'Jules Bailey (born November, 1979) is a former Multnomah County Commissioner who now works at the Oregon Beverage Recycling Cooperative.', 'Early life and education \nBailey was raised in Portland, Oregon and graduated from Lincoln High School.', "He earned a bachelor's degree in  Lewis & Clark College and received MPA/URP from Princeton University\n\nBailey studied in a dual-degree graduate program at Princeton University's Woodrow Wilson School of Public and International Affairs.", 'In January 2017, he began working for the Oregon Beverage Recycling Cooperative as the chief stewardship officer.']


{'ID': 11,
 'indexes': [1, 3, 0, 2, 4, 5],
 'sentences': ['The ship was sunk in 1940.',
  'The ships displaced  at (standard) load and  at deep load.',
  "Orage was a  (torpilleur d'escadre) built for the French Navy during the 1920s.",
  'Design and description\nThe Bourrasque class had an overall length of , a beam of , and a draft of .',
  'They were powered by two geared steam turbines, each driving one propeller shaft, using steam provided by three du Temple boilers.',
  'The turbines were designed to produce , which would propel the ship at .']}

In [ ]:
avg_len = 0
for i, parag in enumerate(train_set):
  parag_len = 0
  for sent in parag["sentences"]:
    parag_len +=len(sent)
  if (i % 10000 == 0):
    print(parag_len)

  avg_len += float(parag_len) * 1/(len(train_set))
print(avg_len)

In [ ]:
bigger_than_512

521463

In [ ]:
nums = 0
for l in range(5):
  for k in range(l, 5):
    for r in range(k + 1, 6):
      print((l, k), (k+1, r))
      nums += 1
print(nums)

(0, 0) (1, 1)
(0, 0) (1, 2)
(0, 0) (1, 3)
(0, 0) (1, 4)
(0, 0) (1, 5)
(0, 1) (2, 2)
(0, 1) (2, 3)
(0, 1) (2, 4)
(0, 1) (2, 5)
(0, 2) (3, 3)
(0, 2) (3, 4)
(0, 2) (3, 5)
(0, 3) (4, 4)
(0, 3) (4, 5)
(0, 4) (5, 5)
(1, 1) (2, 2)
(1, 1) (2, 3)
(1, 1) (2, 4)
(1, 1) (2, 5)
(1, 2) (3, 3)
(1, 2) (3, 4)
(1, 2) (3, 5)
(1, 3) (4, 4)
(1, 3) (4, 5)
(1, 4) (5, 5)
(2, 2) (3, 3)
(2, 2) (3, 4)
(2, 2) (3, 5)
(2, 3) (4, 4)
(2, 3) (4, 5)
(2, 4) (5, 5)
(3, 3) (4, 4)
(3, 3) (4, 5)
(3, 4) (5, 5)
(4, 4) (5, 5)
35
